In [1]:
# 3.0 LLMs and Chat Models

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
import os
# .env 파일 쓰려면 dotenv 사용하기
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = OpenAI(openai_api_key=openai_api_key)
chat = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)
a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")
a, b

('\n\nAs of June 2020, there are eight planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight recognized planets in our solar system. They are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate among astronomers regarding the classification of Pluto as a planet, as it was reclassified as a dwarf planet in 2006 by the International Astronomical Union (IAU).')

In [2]:
# 3.1 Predict Messages

from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian."
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 18.000 chilometri.')

In [3]:
# 3.2 Prompt Templates

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os
# .env 파일 쓰려면 dotenv 사용하기
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

prompt = template.format(country_a="Mexcio", country_b="Thailand")

chat.predict(prompt)

template_message = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template_message.format_messages(
    language="Greek",
    name="Socrates",
    country_a = "Mexico",
    country_b = "korea",
)

chat.predict_messages(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [8]:
# 3.3 OutputParser and LCEL

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content) # ['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto']

chain = template | chat | CommaOutputParser() # | 로 chain 형성!

chain.invoke({
    "max_items":5,
    "question": "What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [14]:
# 3.4 Chaining Chains

from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", ""), 
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."), 
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain # Runnable map

final_chain.invoke({
    "cuisine": "indian"
})

That's great! Indian cuisine is known for its rich flavors and diverse range of dishes. Here's a simple recipe for you to try:

Butter Chicken (Murgh Makhani):
Ingredients:
- 500g boneless chicken, cut into pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 teaspoons ginger-garlic paste
- 2 teaspoons red chili powder
- 1 teaspoon turmeric powder
- 1 teaspoon garam masala
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh coriander leaves for garnish

Instructions:
1. Heat butter in a pan over medium heat. Add the chopped onions and sauté until golden brown.
2. Add ginger-garlic paste and sauté for another minute.
3. Add red chili powder, turmeric powder, and garam masala. Mix well and cook for a minute.
4. Add tomato puree and cook for 5-7 minutes until the oil separates from the mixture.
5. Add the chicken pieces and cook until they are cooked through and tender.
6. Stir in the heavy cream and season with salt. Simmer for another 5 minutes.
7. Garnish with fre

AIMessageChunk(content="Butter Chicken, also known as Murgh Makhani, is a popular Indian dish that is traditionally made with chicken. However, as a vegetarian chef, I can suggest a delicious alternative to replace the chicken in this recipe.\n\nInstead of using chicken, you can use paneer, which is a type of Indian cheese. Paneer has a mild and creamy flavor that works well in rich and flavorful dishes like Butter Chicken. Here's how you can modify the recipe:\n\nIngredients:\n- 500g paneer, cut into cubes\n- 2 tablespoons butter\n- 1 onion, finely chopped\n- 2 teaspoons ginger-garlic paste\n- 2 teaspoons red chili powder\n- 1 teaspoon turmeric powder\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1/2 cup heavy cream\n- Salt to taste\n- Fresh coriander leaves for garnish\n\nInstructions:\n1. Heat butter in a pan over medium heat. Add the chopped onions and sauté until golden brown.\n2. Add ginger-garlic paste and sauté for another minute.\n3. Add red chili powder, turmeric powder

In [2]:
# 4.1 FewShotPromptTemplate

from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {"question": "What do you know about France?", 
             "answer": """
             Here is what I know: 
             Capital: Paris 
             Language: French 
             Food: Wine and Cheese 
             Currency: Euro
             """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
            """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]


example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [3]:
# 4.2 FewShotChatMessagePromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum Soup
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum Soup\n        Currency: Thai Baht\n        ')

In [4]:
# 4.3 LengthBasedExampleSelector

from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [5]:
# 4.4 Serialization and Composiotion

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg! Me favorite food be a good ol' plate o' fish and chips! The salty sea air makes me crave the taste of fresh fish, battered and fried to perfection. And don't forget the crispy golden chips to go along with it! It be a meal fit for a pirate like meself! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate o' fish and chips! The salty sea air makes me crave the taste of fresh fish, battered and fried to perfection. And don't forget the crispy golden chips to go along with it! It be a meal fit for a pirate like meself! Arg arg!")

In [6]:
# 4.5 Caching

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
set_debug(True)

set_llm_cache(SQLiteCache("cache.db"))

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- 1/2 teaspoon of salt
- Water (if needed)

Here's a step-by-step guide to making Italian pasta:

1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.
2. Crack the eggs into the well and add the salt.
3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.
4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. If the dough is too dry, add a little water, one tablespoon at a time, until it comes together. If it's too sticky, add a little flour.
5. Knead the dough for about 5-10 minutes until it becomes smooth and elastic.
6. Shape the dough into a ball and cover it with a clean kitchen towel. Let it 

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. If the dough is too dry, add a little water, one tablespoon at a time, until it comes together. If it's too sticky, add a little flour.\n5. Knead the dough for about 5-10 minutes until it becomes smooth and elastic.\n6. Shape the dough into a ball and cover it with a clean kitchen towel. Let it rest for at least 30 minutes to allow the gluten to relax.\n7. After resting, divide the dough into smaller por

In [3]:
# 4.6 Serialization

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.callbacks import get_openai_callback
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

chat = ChatOpenAI(openai_api_key=openai_api_key, 
                  temperature=0.1, 
                  streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler()])


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)

chat2 = OpenAI(openai_api_key=openai_api_key, 
               temperature=0.1, 
               max_tokens=450, 
               model="gpt-3.5-turbo-16k")

chat2.save("model.json")

chat3 = load_llm("model.json")

Here is a simple recipe for making soju at home:

Ingredients:
- 1 cup of rice
- 1 cup of nuruk (Korean fermentation starter)
- 8 cups of water
- 1 tablespoon of yeast
- 1 cup of sugar (optional, for sweetening)

Instructions:
1. Rinse the rice thoroughly until the water runs clear.
2. Soak the rice in water for about 1 hour, then drain.
3. Steam the rice until it becomes soft and fully cooked.
4. Let the cooked rice cool down to room temperature.
5. In a large container, combine the cooled rice, nuruk, and water. Mix well.
6. Cover the container with a clean cloth and let it ferment for about 7-10 days at room temperature.
7. After the fermentation period, strain the mixture to remove any solids.
8. Dissolve the yeast in a small amount of warm water and add it to the strained liquid.
9. Cover the container again and let it ferment for an additional 2-3 days.
10. After the second fermentation, strain the liquid again to remove any remaining solids.
11. At this point, you can add sugar 

/home/rlagusrb/miniconda3/envs/FullStack-GPT/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/rlagusrb/miniconda3/envs/FullStack-GPT/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
# 5.0 Conversation Buffer Memory

from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

model = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [7]:
# 5.1 Conversation Buffer Window Memory

from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"inputs": input}, {"output": output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
add_message(5, 5) # 1번 메시지는 삭제!

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [ ]:
# 5.2 Conversation Summary Memory
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"inputs": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!!!")

get_history()

In [ ]:
# 5.3 Conversation Summary Buffer Memory

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"inputs": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!!!")

In [ ]:
# 5.4 Conversation KG Memory

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationKGMemory
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"inputs": input}, {"output": output})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})

In [1]:
# 5.5 Memory on LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and bustling city. How can I assist you today, Nico?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'

In [3]:
# 5.6 Chat Based Memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Hello Nico! How can I assist you today?'

In [ ]:
# 5.7 LCEL Based Memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("My name is nico")
invoke_chain("What is my name?")